#### Metrocuadrado.com Web Data Scrapping

This example was created to show how to do web scrapping when headers are needed. For more details and tutorials go to: www.squaid.com

In [21]:
# Author www.squaid.com
# Import libraries we are going to use
import pandas as pd
from datetime import date
import itertools
import requests
import json
import re

In [ ]:
# Just a function needed
def get_mcdata(business: str, property_type: str, city: list, limit: int) -> list:
    """This functions is created to ger data from Finca Raiz a bit improved in order to improve amount of data scrapped"""
    initial_limit = limit
    offset = 0
    max_hits = 10000
    api_key = 'P1MfFHfQMOtL16Zpg36NcntJYCLFm8FqFfudnavl'

    url_mc = f'https://www.metrocuadrado.com/rest-search/search?realEstateBusinessList={business}&city={city}&realEstateTypeList={property_type}&from={offset}&size={limit}'
    headers = {'X-Api-Key':api_key}

    data = []
    response = requests.get(url=url_mc, headers=headers)
    response_body =  json.loads(response.text)

    total_assets = response_body['totalHits']
    total_results = response_body['totalEntries']
    data_response = response_body['results']

    data.append(data_response)

    if total_results > len(data_response):

        if total_assets < max_hits:
            while len(data_response) > 0:
                if offset + limit <= total_results:
                    offset += limit
                else:
                    offset += limit
                    limit = total_results - offset

                url_mc = re.sub('from=(\d+)',f'from={offset}',url_mc)
                response = requests.get(url=url_mc, headers=headers)
                response_body =  json.loads(response.text)
                data_response = response_body['results']
                data.append(data_response)
        else:
            
            for stratum_id in ['stratumList=1','stratumList=2','stratumList=3','stratumList=4','stratumList=5','stratumList=6','stratumList=7','stratumList=8']:

                offset = 0
                limit = initial_limit

                url_mc = f'https://www.metrocuadrado.com/rest-search/search?realEstateBusinessList={business}&city={city}&realEstateTypeList={property_type}&from={offset}&size={limit}&{stratum_id}'
                headers = {'X-Api-Key':api_key}
                response = requests.get(url=url_mc, headers=headers)
                response_body =  json.loads(response.text)

                total_assets = response_body['totalHits']
                total_results = response_body['totalEntries']
                data_response = response_body['results']

                data.append(data_response)

                while (len(data_response) > 0) & (offset + limit < max_hits):
                    if offset + limit <= total_results:
                        offset += limit
                    else:
                        offset += limit
                        limit = total_results - offset

                    url_mc = re.sub('from=(\d+)',f'from={offset}',url_mc)
                    response = requests.get(url=url_mc, headers=headers)
                    response_body =  json.loads(response.text)
                    data_response = response_body['results']
                    data.append(data_response)

    data = list(itertools.chain(*data))
    return data

In [ ]:

# Let´s get data Colombia Real State
business_types = ["venta","arriendo"]
property_types = ['apartamento','casa','oficina','local','bodega','lote','finca','edificio-de-oficinas','consultorio','edificio-de-apartamentos']
cities = {
    'cali': 'Cali',
    'jamundi': 'Jamundi',
    'bogota': 'Bogot%C3%A1',
    'medellin': 'Medell%C3%ADn',
    'cota': 'Cota%20(Incluye%20Siberia)',
    'chia': 'Ch%C3%ADa',
    'girardot': 'Girardot',
    'santa_marta': 'Santa%20Marta',
    'cartagena': 'Cartagena%20de%20Indias'
}
limit = 250

try:
    print(isinstance(assets,dict))
except:
    assets = {}

for city in cities:
    if not(city in assets.keys()):
        assets[city] = {}
    for business in business_types:
        if not(business in assets[city].keys()):
            assets[city][business] = {}
        for type in property_types:
            print(f"City: {city} Type:{type} ...")
            data = get_mcdata(business, type, cities[city], limit) 
            print(f"... Data recolected:{len(data)}")
            assets[city][business][type] = data


In [ ]:
# Save raw data
today = date.today()
d1 = today.strftime("%d%m%Y")
with open(f"./data/mc_{d1}.json", "w") as archivo:
    json.dump(assets, archivo)

In [ ]:
#Convert data into dataframe
data_full_df = []
for city_key in assets.keys():
    for business_key in assets[city_key].keys():
        for type_key in assets[city_key][business_key].keys():
            data_df = pd.DataFrame(assets[city_key][business_key][type_key])
            data_df["city"] = city_key
            data_df["business"] = business_key
            data_df["type"] = type_key
            data_full_df.append(data_df)

datafull = pd.concat(data_full_df)

In [ ]:

# Save cleaned data csv
today = date.today()
d1 = today.strftime("%d%m%Y")
datafull.to_csv(f"./data/mc_cleaned_{d1}.csv", "w")

In [ ]:
datafull